In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import torch

sys.path.append(os.path.join(os.getenv("HOME"), "RNN_Manifold/"))
import s1_direct_product_decoder, s1_direct_product_generator, geometry_util

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Generate some ring data

In [3]:
encoder, decoder = s1_direct_product_generator.train(1, 12, device, n_training_iterations=3000)
angles = np.arange(start=0, stop=2 * np.pi, step=0.01)
with torch.no_grad():
    points = geometry_util.torch_angles_to_ring(torch.tensor(angles, dtype=torch.get_default_dtype()).to(device))
    points = torch.unsqueeze(points, -2)
    ring_embedded_points = encoder(points)
ring_embedded_points = ring_embedded_points.cpu().numpy()

/home/trevor/brainvenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


iteration: 0, decoding loss: 0.9553124904632568, distance cost: 0.009668017737567425
iteration: 1, decoding loss: 0.5222339630126953, distance cost: 0.025882022455334663
iteration: 2, decoding loss: 0.34629884362220764, distance cost: 0.04498959705233574
iteration: 3, decoding loss: 0.22818253934383392, distance cost: 0.06924714893102646
iteration: 4, decoding loss: 0.19044853746891022, distance cost: 0.0845930203795433
iteration: 5, decoding loss: 0.16612711548805237, distance cost: 0.08098747581243515
iteration: 6, decoding loss: 0.11563632637262344, distance cost: 0.060826484113931656
iteration: 7, decoding loss: 0.07695960998535156, distance cost: 0.02846253663301468
iteration: 8, decoding loss: 0.061283450573682785, distance cost: 0.0062489896081388


KeyboardInterrupt: 

In [ ]:
sample_indicies = np.random.choice(np.shape(ring_embedded_points)[0], 100)
test_ring_data = ring_embedded_points[sample_indicies, :]
test_ring_phases = angles[sample_indicies]

Form a simple torus as the product of two rings

In [ ]:
possible_samples = np.arange(start=0, stop=np.shape(test_ring_data)[0], step=1)
possible_pairs = []
for sample_1 in possible_samples:
    for sample_2 in possible_samples:
        possible_pairs.append([sample_1, sample_2])

possible_pairs = np.array(possible_pairs)
n_samples = 1000
samples = np.random.choice(len(possible_pairs), n_samples)
indicies = possible_pairs[samples, :]

data_1 = test_ring_data[indicies[:, 0], :]
phases_1 = test_ring_phases[indicies[:, 0]]

data_2 = test_ring_data[indicies[:, 1], :]
phases_2 = test_ring_phases[indicies[:, 1]]

product_torus_data = np.concatenate([data_1, data_2], axis=1)
product_torus_data = product_torus_data/np.mean(np.abs(product_torus_data))
product_torus_phases = np.stack([phases_1, phases_2], axis=-1)

Decode the product torus

In [ ]:
encoder, decoder = s1_direct_product_decoder.train(data=product_torus_data, manifold_dim=2, device=device,
                                                   n_training_iterations=3000, decoder_weight=10, order_red_weight=0.1)

In [ ]:
with torch.no_grad():
    product_predicted_phases = decoder(torch.tensor(product_torus_data, dtype=torch.get_default_dtype()).to(device))[1].cpu().numpy()

In [ ]:
def reference_phases(phases):
    phases_refd = phases - phases[0]
    phases_refd = np.arctan2(np.sin(phases_refd), np.cos(phases_refd))
    return phases_refd * np.sign(phases_refd[1])

In [ ]:
def compare_to_ground_truth(predicted_phases, ground_truth_phases, plot_axes):
    refd_test_phases = reference_phases(predicted_phases)
    refd_true_phases = reference_phases(ground_truth_phases)
    if np.abs(refd_test_phases[1,1] - refd_true_phases[1, 0]) < np.abs(refd_test_phases[1,1] - refd_true_phases[1, 1]):
        refd_test_phases = np.stack([refd_test_phases[:, 1], refd_test_phases[:, 0]], axis=-1)
    line = np.arange(start=-np.pi, stop=np.pi, step=0.01)
    plot_axes[0].scatter(refd_true_phases[:, 0], refd_test_phases[:, 0])
    plot_axes[0].plot(line, line, color="black", linestyle="--", label="y=x")
    plot_axes[1].scatter(refd_true_phases[:, 1], refd_test_phases[:, 1])
    plot_axes[1].plot(line, line, color="black", linestyle="--", label="y=x")
    plot_axes[0].set_xlabel("True Phase")
    plot_axes[0].set_title("Phase 1")
    plot_axes[1].set_xlabel("True Phase")
    plot_axes[1].set_title("Phase 2")
    plot_axes[0].set_ylabel("Found Phase")
    return refd_test_phases, refd_true_phases

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True)
referenced_product_predicted_phases, referenced_product_true_phases = compare_to_ground_truth(product_predicted_phases, product_torus_phases, axs)

Make a torus directly using the generative model

In [ ]:
encoder, decoder = s1_direct_product_generator.train(2, 24, device, n_training_iterations=5000)

torus_phases = np.random.uniform(0, 2 * np.pi, (1000, 2))
with torch.no_grad():
    points = geometry_util.torch_angles_to_ring(torch.tensor(torus_phases, dtype=torch.get_default_dtype()).to(device))
    torus_embedded_points = encoder(points)
torus_embedded_points = torus_embedded_points.cpu().numpy()

Decode the generated torus

In [ ]:
normed_torus_embedded_points = torus_embedded_points/np.mean(np.abs(torus_embedded_points))

In [ ]:
encoder, decoder = s1_direct_product_decoder.train(data=normed_torus_embedded_points, manifold_dim=2, device=device,
                                                   n_training_iterations=3000, decoder_weight=10, order_red_weight=0.1)


In [ ]:
with torch.no_grad():
    gen_predicted_phases = decoder(torch.tensor(normed_torus_embedded_points, dtype=torch.get_default_dtype()).to(device))[1].cpu().numpy()

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True)
referenced_gen_predicted_phases, referenced_gen_true_phases = compare_to_ground_truth(gen_predicted_phases, torus_phases, axs)
